In [11]:
!pip install -q -q -q geopandas
!pip install -q -q -q rtree

In [1]:
import geopandas as gpd
import shutil

In [2]:
shutil.unpack_archive('paris.zip')

Chargement des données carroyées et des quartiers de la politique de ville sur Paris

In [29]:
carreaux = gpd.read_file('data/paris.shp')

In [68]:
variables = ["Ind","Men","Men_pauv","Men_1ind","Men_5ind","Men_prop","Men_fmp","Ind_snv",
             "Men_surf","Men_coll","Men_mais","Log_av45","Log_45_70","Log_70_90","Log_ap90",
             "Log_inc","Log_soc","Ind_0_3","Ind_4_5","Ind_6_10","Ind_11_17","Ind_18_24",
             "Ind_25_39","Ind_40_54","Ind_55_64","Ind_65_79","Ind_80p","Ind_inc"]

In [64]:
carreaux = carreaux[["IdINSPIRE", "geometry"] + variables]

In [65]:
qpv = gpd.read_file('data/qp_bdtopo_com_75056_2020.shp')

### Intersection spatiale entre les deux données spatiales (interpolation spatiale simple)

In [58]:
intersection = gpd.sjoin(qpv, carreaux, how='inner', op='intersects')

In [59]:
intersection.head(3)

,code,libelle,geometry,index_right,IdINSPIRE,Ind,Men,Men_pauv,Men_1ind,Men_5ind,...,Ind_4_5,Ind_6_10,Ind_11_17,Ind_18_24,Ind_25_39,Ind_40_54,Ind_55_64,Ind_65_79,Ind_80p,Ind_inc
0,QP075001,B?dier - Boutro,"POLYGON ((653957.500 6857921.700, 653802.699 6...",133,CRS3035RES200mN2885200E3761600,8.0,3.7,0.9,1.6,0.3,...,0.1,0.4,0.4,0.5,2.2,1.7,0.8,0.8,0.3,0.2
0,QP075001,B?dier - Boutro,"POLYGON ((653957.500 6857921.700, 653802.699 6...",559,CRS3035RES200mN2885200E3761800,520.0,194.0,34.0,59.0,28.0,...,13.0,39.5,56.0,39.0,95.0,97.0,62.0,53.0,36.0,8.0
0,QP075001,B?dier - Boutro,"POLYGON ((653957.500 6857921.700, 653802.699 6...",643,CRS3035RES200mN2885200E3762000,244.0,120.0,69.0,81.0,14.0,...,8.0,22.0,20.0,42.0,77.0,35.0,12.0,8.0,7.0,0.0


Aggrégation des données par qpv

In [26]:
qpv_statistics = intersection.dissolve(by=['code', 'libelle'], aggfunc='sum', as_index=False)

In [27]:
qpv_statistics.head(3)

,code,libelle,geometry,index_right,Ind,Men,Men_pauv,Men_1ind,Men_5ind,Men_prop,...,Ind_4_5,Ind_6_10,Ind_11_17,Ind_18_24,Ind_25_39,Ind_40_54,Ind_55_64,Ind_65_79,Ind_80p,Ind_inc
0,QP075001,B?dier - Boutro,"POLYGON ((653957.500 6857921.700, 653802.699 6...",34133,5112.0,2150.7,444.9,889.6,237.3,141.8,...,175.6,365.4,498.9,294.0,1175.2,1126.7,494.8,394.8,173.3,96.2
1,QP075002,Chaufourniers,"POLYGON ((653876.162 6864327.010, 653913.377 6...",6625,10886.5,5048.0,955.0,2335.0,385.0,1482.0,...,242.0,570.0,822.0,545.5,2693.0,2265.0,1291.0,1276.0,445.0,257.5
2,QP075003,Villa d'Este,"POLYGON ((653491.700 6858200.401, 653559.901 6...",1794,4225.5,1505.0,378.0,417.0,251.0,403.0,...,128.0,284.5,426.0,266.5,895.0,953.0,461.0,362.0,138.0,74.0


### Intersection spatiale entre les deux données spatiales (au prorata de la surface)

In [88]:
intersection = gpd.overlay(qpv, carreaux)

In [89]:
ponderation = intersection.area / 200**2

In [90]:
intersection.loc[:,variables] = intersection.loc[:,variables].mul(ponderation, axis=0)

Aggrégation des données par qpv

In [91]:
qpv_statistics = intersection[['code', 'libelle', 'geometry'] + variables].dissolve(by=['code', 'libelle'], aggfunc='sum', as_index=False)

In [92]:
qpv_statistics.head(3)

,code,libelle,geometry,Ind,Men,Men_pauv,Men_1ind,Men_5ind,Men_prop,Men_fmp,...,Ind_4_5,Ind_6_10,Ind_11_17,Ind_18_24,Ind_25_39,Ind_40_54,Ind_55_64,Ind_65_79,Ind_80p,Ind_inc
0,QP075001,B?dier - Boutro,"POLYGON ((653957.500 6857921.700, 653882.944 6...",2263.586725,947.723168,183.294871,386.401354,103.203330,10.150395,141.014309,...,77.003176,156.633659,221.835078,124.020096,489.731878,504.703615,232.002095,193.528884,84.349484,43.384613
1,QP075002,Chaufourniers,"POLYGON ((654071.916 6864111.407, 654060.266 6...",2059.521112,939.802885,188.783286,414.347185,68.450917,252.427838,106.959164,...,43.689349,109.589950,156.776529,114.155646,481.115702,431.814637,239.187508,250.425067,89.673315,48.853005
2,QP075003,Villa d'Este,"POLYGON ((653666.517 6858142.994, 653674.700 6...",930.244592,328.540445,113.683012,91.519897,54.756895,157.325714,21.574052,...,20.175748,47.777636,76.144552,68.791819,186.211971,205.738699,126.222610,107.047370,42.578974,9.026120
